<a href="https://colab.research.google.com/github/TahaErr/Civil_Rag/blob/main/Civil_Rag_Fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate bitsandbytes
!pip install sentence-transformers
!pip install faiss-cpu
!pip install langchain


In [ ]:
!pip install optimum

In [ ]:
!pip install gptqmodel

# Mistral 7B Modelinin Yüklenmesi (HuggingFace üzerinden)

---





In [ ]:
from huggingface_hub import login

login()


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import optimum

model_id = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16
)


In [ ]:
!pip install PyMuPDF


In [ ]:
from google.colab import files

uploaded = files.upload()


In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

pdf_text = extract_text_from_pdf("/content/224R_01.pdf")  # yüklediğin dosya adı
print(pdf_text[:1000])  # ilk 1000 karakteri yazdır


In [ ]:
!pip install langchain sentence-transformers faiss-cpu


In [ ]:
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter

# Metni LangChain belgesine dönüştür
docs = [Document(page_content=pdf_text)]

# Parçalama işlemi (300 karakterlik parçalar, 50 karakter üst üste binecek şekilde)
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)

print(f"{len(split_docs)} adet parça oluşturuldu.")
print("İlk parça:\n", split_docs[0].page_content)


In [ ]:
!pip install -U langchain-community

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# HuggingFace tabanlı embedding modeli
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# FAISS veritabanını oluştur
vectorstore = FAISS.from_documents(split_docs, embedding_model)

print("Embedding ve FAISS veritabanı başarıyla oluşturuldu.")


In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
# Example question in English
user_question = "What are the main causes of cracking in concrete?"

# Search relevant chunks from FAISS
retrieved_docs = vectorstore.similarity_search(user_question, k=1)

# Merge them into context
context = "\n\n".join([doc.page_content for doc in retrieved_docs])


In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = """
You are a civil engineer. Based on the following context, answer the question clearly and technically.


Question:
What are the main causes of cracking in concrete?

Answer:
"""

response = generator(prompt, max_new_tokens=300, do_sample=True, temperature=0.7, top_p=0.9)[0]['generated_text']
print(response)


In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

def generate_answer(question_text, max_tokens=300):
    prompt = f"""
You are a civil engineer. Based on the following context, answer the question clearly and technically.

Question:
{question_text}

Answer:
"""
    response = generator(
        prompt,
        max_new_tokens=max_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )[0]['generated_text']
    return response


In [ ]:
question = "What are the main causes of cracking in concrete?"
print(generate_answer(question))


In [ ]:
!pip install gradio


In [ ]:
import gradio as gr
from transformers import pipeline

# Hazır model pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Fonksiyonumuz
def generate_answer(question_text, max_tokens=300):
    prompt = f"""
You are a civil engineer. Based on the following context, answer the question clearly and technically.

Question:
{question_text}

Answer:
"""
    response = generator(
        prompt,
        max_new_tokens=max_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )[0]['generated_text']
    return response

# Gradio arayüzü
demo = gr.Interface(
    fn=generate_answer,
    inputs=gr.Textbox(lines=2, placeholder="Enter your technical question about concrete..."),
    outputs="text",
    title="RAG-based Civil Engineering Assistant",
    description="Ask your question about concrete behavior, cracking, or construction — powered by Mistral 7B"
)

# Başlat
demo.launch()
